In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('wb_bus_details.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/bolpur-west-bengal-to-kolkata: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF75ECBEEA2+31554]
	(No symbol) [0x00007FF75EC37ED9]
	(No symbol) [0x00007FF75EAF872A]
	(No symbol) [0x00007FF75EB48434]
	(No symbol) [0x00007FF75EB4853C]
	(No symbol) [0x00007FF75EB8F6A7]
	(No symbol) [0x00007FF75EB6D06F]
	(No symbol) [0x00007FF75EB8C977]
	(No symbol) [0x00007FF75EB6CDD3]
	(No symbol) [0x00007FF75EB3A33B]
	(No symbol) [0x00007FF75EB3AED1]
	GetHandleVerifier [0x00007FF75EFC8B1D+3217341]
	GetHandleVerifier [0x00007FF75F015AE3+3532675]
	GetHandleVerifier [0x00007FF75F00B0E0+3489152]
	GetHandleVerifier [0x00007FF75ED6E776+750614]
	(No symbol) [0x00007FF75EC4375F]
	(No symbol) [0x00007FF75EC3EB14]
	(No symbol) [0x00007FF75EC3ECA2]
	(No symbol) [0x00007FF75EC2E16F]
	BaseThreadInitThunk [0x00007FFE5633257D+29]
	RtlUserThreadStart [0x00007FFE572CAF28+40]

Error occurred while scraping bus details for https://www.redbus

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),05:00,03h 30m,08:30,4.1,INR 150,38 Seats available
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),05:15,04h 00m,09:15,4.2,INR 155,24 Seats available
2,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),05:30,03h 30m,09:00,3.7,INR 150,47 Seats available
3,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),06:00,03h 30m,09:30,2.9,INR 150,44 Seats available
4,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),06:15,04h 00m,10:15,4.0,INR 155,19 Seats available
...,...,...,...,...,...,...,...,...,...,...
763,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,04h 20m,11:15,4.9,INR 149,45 Seats available
764,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,WBTC (CTC) HABRA-DIGHA via Bally - 26|13:00,Non AC Seater (2+3),13:00,04h 25m,17:25,4.2,INR 149,51 Seats available
765,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,Santosh Bus Service,A/C Seater Push Back (2+3),05:10,04h 20m,09:30,4.6,360,4 Seats available
766,Habra to Midnapore,https://www.redbus.in/bus-tickets/habra-to-mid...,WBTC (CTC) Habra-Midnapur - 194|05:55,Non AC Seater (2+3),05:55,04h 20m,10:15,4.1,INR 138,19 Seats available
